In [15]:
import gensim
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

In [42]:
#le data frame est saved à partir de Help_rapport 
data_cleaned = pd.read_csv("data/data_cleaned_100k.csv").fillna("")
data_cleaned

,Categorie1,Description
0,INFORMATIQUE,batter acer aspir one ion mah noir compatibl b...
1,TELEPHONIE - GPS,coqu rigid bleu lagon alcatel motif drapeau li...
2,TELEPHONIE - GPS,facad coqu cellular lin shckgal minip marqu ag...
3,TELEPHONIE - GPS,coqu meteor tpu nexus
4,TELEPHONIE - GPS,coqu soupl transparent flex motif keep calm pl...
...,...,...
99995,DECO - LINGE - LUMINAIRE,souvenir franc tour eiffel miniatur prestig co...
99996,LIBRAIRIE,fast track wast fre manufacturing john dav fas...
99997,TELEPHONIE - GPS,fitbag bong giraf houss pochet telephon portab...
99998,DECO - LINGE - LUMINAIRE,grand tableau minn


In [43]:
## On s'est rendu compte que le dernier token était "" pour chaque produit
train_array_token = [line.split(' ') for line in data_cleaned.Description.values]
print(train_array_token[:5])

[['batter', 'acer', 'aspir', 'one', 'ion', 'mah', 'noir', 'compatibl', 'batter', ''], ['coqu', 'rigid', 'bleu', 'lagon', 'alcatel', 'motif', 'drapeau', 'liberi', 'film', 'coqu', 'rigid', 'ultra', 'fin', 'bleu', 'lagon', 'original', 'muzzano', 'motif', 'drapeau', 'liberi', 'alcatel', ''], ['facad', 'coqu', 'cellular', 'lin', 'shckgal', 'minip', 'marqu', 'agree', 'samsungmobil', 'compatibl', 'galaxy', 'minimatier', 'caoutchouc', 'soupl', ''], ['coqu', 'meteor', 'tpu', 'nexus', ''], ['coqu', 'soupl', 'transparent', 'flex', 'motif', 'keep', 'calm', 'play', 'football', 'coqu', 'soupl', 'ultra', 'fin', 'transparent', 'original', 'muzzano', 'motif', 'keep', 'calm', 'play', 'footbal', '']]


In [44]:
Features_dimension = 500
hs = 0
negative = 10

In [45]:
#augmenter min_count

sg = 1
print("Start learning skip-gram Word2Vec")
ts = time.time()
model_sg = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)

Start learning skip-gram Word2Vec
Learning time : 96.86 seconds Word2Vec


In [46]:
#utiliser une moyenne pondérée TFIDF (moyenne qui donne un poids à chq mot)
#+le mot apparait de fois dans le document, - il aura d'importance

#fonction qui crée la list de features

def set_features(array_token_prod,dim_feature,model):
    features= []
    #on prend les tokens de chaque produit
    for token_prod in array_token_prod:
        #on crée le np final du produit
        feat_prod= np.zeros(dim_feature)
        # on prend chaque token dans la desc prod
        for token in token_prod:
            #on somme les vecteurs et on divise direct par le nombre de token
            feat_prod +=model[token]/len(token_prod)
            
        #on rajoute le vectuer moyenné produit dans la list features
        features += [feat_prod]
    return features

In [47]:
#init notre list de features
features=set_features(train_array_token,Features_dimension,model_sg)

<ipython-input-46-52aca7a56739>:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feat_prod +=model[token]/len(token_prod)


In [48]:
len(features) 

100000

In [49]:
type(features)

list

In [50]:
import pickle
with open("features100k_word2vec.txt", "wb") as fp:   
    pickle.dump(features, fp)

On a bien 100 000 vecteurs colonne de dim 300

In [9]:
#on va split comme précédemment avec ces nvx features et label
label = data_cleaned["Categorie1"].values
X_train, X_test, y_train, y_test = train_test_split(features,label, test_size=0.1, random_state=42)

Procédure habituelle ensuite..

In [10]:
ts = time.time()
lr = LogisticRegression()
lr.fit(X_train, y_train)
te = time.time()
print(te-ts, "sec")

/Users/paulbrunet/anaconda3/envs/Projet4A/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


789.879961013794 sec


In [11]:
print("Score apprentissage ",lr.score(X_train,y_train))
print("Score test ",lr.score(X_test,y_test))

Score apprentissage  0.9030055555555555
Score test  0.90134


### Autre modèle:

In [ ]:
sg = 0
print("Start learning CBOW Word2Vec")
ts = time.time()
model_cbow = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)

In [ ]:
features2=set_features(train_array_token,Features_dimension,model_cbow)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2,label, test_size=0.1, random_state=42)

In [ ]:
ts = time.time()
lr2 = LogisticRegression()
lr2.fit(X_train2, y_train2)
te = time.time()
print(te-ts, "sec")

In [ ]:
print("Score apprentissage ",lr2.score(X_train2,y_train2))
print("Score test ",lr2.score(X_test2,y_test2))

## Exploration utilisation modèle/token

In [ ]:
model_sg.most_similar(["montr"])

In [ ]:
model_cbow.most_similar(["montr"])

In [ ]:
model_sg["montr"].shape

In [ ]:
#faire tourner sg sur le max de données avec données nettoyées
#puis prediction avec la regression à comparer le mm modele avec les features de count vectorizer

# Extra Work

pre-trained network


Github du projet[https://github.com/Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors) appris sur 1Giga d'articles de wikipedia en mode **Skip-Gram*

Vous pouvez télécharger ce modèle en suivant ce [lien](https://drive.google.com/file/d/0B0ZXk88koS2KM0pVTktxdG15TkE/view). Dezipez-le puis téléchargez le modèle en indiquant la direction du fichier "fr/bin"

In [ ]:
model_online_dir = "data/fr/fr.bin"
#model_online_dir = "ACOMPLETER/fr.bin"
model_online = gensim.models.Word2Vec.load(model_online_dir)